# Text generation models

One very basic kind of text generation model is the Markov model.  In
such a model, we have a state which consists of the previous character.
We also have a matrix of transitions from one character to another.  We
*train* the model by feeding it some text, and observing the
transitions.  We can then generate more text from the model.

In [ ]:
from collections import defaultdict
import re

def train_sentence(transitions, sentence, depth=1):
    # We need a "special" character to represent the beginning of a sentence.
    # This is also the character we'll use to feed the generator, below.
    prevchar = "•" * depth
    for char in sentence:
        transitions[prevchar][char] += 1
        prevchar = prevchar[1:] + char

    return transitions

def split_text(text):
    for sentence in re.finditer(".*?([.?!][”’]?|\n\n)", text, re.DOTALL):
        # Turn all sequences of whitespace into a single space
        sentence = re.sub("[ \t\n\r]+", " ", sentence.group(0)).strip()
        if sentence != '':
            yield sentence

def train(filename, depth=1):
    transitions = defaultdict(lambda: defaultdict(int))
    with open(filename) as fin:
        text = fin.read()
        for sentence in split_text(text):
            if len(sentence) < 3:
                continue
            transitions = train_sentence(transitions, sentence, depth)

    return transitions

In [ ]:
import pandas as pd
import numpy as np

def format_transitions(trs):
    rows = []
    for key in trs:
        for key2 in trs[key]:
            rows.append({'from': key, 'to': key2, 'n': trs[key][key2]})
    data = pd.DataFrame(rows)
    data = data.pivot_table(index='from', columns='to', values='n')
    data = data.div(data.sum(axis=1), axis=0)
    data[np.isnan(data)] = 0

    return data

In [ ]:
def produce(transitions):
    if isinstance(transitions, defaultdict):
        transitions = format_transitions(transitions)

    # Nifty trick: auto-calculate the depth we were trained on
    depth = len(transitions.index[0])

    output = ""
    last = "•" * depth
    nxt = ""

    while nxt not in [".", "?", "!"]:
        trs = transitions.loc[last]
        nxt = np.random.choice(trs.index, p=trs)
        last = last[1:] + nxt
        output += nxt

    return output

In [ ]:
tr = train("alice.txt", 4)

In [ ]:
[produce(tr) for x in range(5)]

Ideas for extension:

- Train on a different text
- Try normalizing the text in different ways (e.g. what happens if you take
  out quotation marks?)
- Play around with different depths.  Do different ones work better for
  different texts?
- Try generating longer passages (you will need to alter the training
  also)

# Hidden Markov Models

In [ ]:
import hmmlearn.hmm

In [ ]:
N_STATES = 16

In the below code, `alphabet` is the list of characters that appear in the text.  `emission_probs` is a matrix with `N_STATES` rows and `len(alphabet)` columns.  It is the initial estimate of the probability of each letter in each state.  As a "dumb" initial guess, we set all states to have the probability of each letter in the data.

In [ ]:
with open("alice.txt") as fin:
    text = fin.read()
    
alphabet = set(list(text))
alphabet -= {"\n"}
alphabet = sorted(alphabet)

frequencies = defaultdict(int)
for char in text:
    if char == '\n':
        continue
    frequencies[char] += 1
    
total_chars = sum([x for x in frequencies.values()])
    
emission_probs = np.stack([np.fromiter([frequencies[x] / total_chars for x in alphabet], dtype=np.float64)] * N_STATES)

Now we need to transform the characters in the text into numbers, because HMMs speak numbers.  We do this manually, but we could just as easily use `sklearn.preprocessing.LabelEncoder` for it.  We need to end up with a numpy vector, each element of which is a one-element vector containing a character-number (that's what the `.reshape()` incantation is for

In [ ]:
sentences = list(split_text(text))
letter_vector = np.fromiter([alphabet.index(char) for sentence in sentences for char in sentence], dtype=np.int64).reshape(-1,1)
lengths = list(map(len, sentences))

In [ ]:
len(letter_vector) - sum(lengths)

In [ ]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=N_STATES, algorithm="viterbi", n_iter=100, verbose=True)

hmm.emission_prob_ = emission_probs

hmm.fit(letter_vector, lengths)

In [ ]:
symbols, _states = hmm.sample(30)

In [ ]:
''.join([alphabet[x[0]] for x in symbols])

Tuning options:
- use words instead of letters
- explore viterbi vs map algorithms

# Neural networks